# Convert raw files into parquet file format 

In [1]:
import warnings
warnings.filterwarnings("ignore")


from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, Row, ArrayType, StringType
import pandas as pd
import json
import datetime
import re

master = 'local'
appName = 'Free time exercise'

spark = SparkSession.builder.appName(appName).getOrCreate()

if spark.sparkContext:
    print('===============')
    print(f'AppName: {spark.sparkContext.appName}')
    print(f'Master: {spark.sparkContext.master}')
    print('===============')
else:
    print('Could not initialise pyspark session')

# df= spark.read.csv("file:///home/ns22/Desktop/rationcard/batch_1_500000_entities_23_dec_2021_rationcard.csv",header=True, sep=",")
df= spark.read.csv("file:///home/ns22/Desktop/rationcard/",header=True, sep=",")
# df1 = df.coalesce(1)
df.count()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/07 12:12:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
AppName: Free time exercise
Master: local[*]


15219768

In [3]:
df.rdd.getNumPartitions()

36

In [4]:
df1 =  df.coalesce(1)

In [5]:
# 3.8 Gb ration card csv files compressed to 1.3 Gb
df1.write.parquet("file:///home/ns22/Desktop/test_ration_parquet_new")

In [65]:
test_df = spark.read.parquet("file:///home/ns22/Desktop/test_ration_parquet_new")

In [66]:
test_df.count()

15219768

In [15]:
salutations = ["mr","miss","dr","srimati","md"]
import re
num_punc = "[^a-zA-z\\s.]"
def clean_name(name):
    if name is not None:
        clean_name = re.sub(num_punc,r"",name.strip().lower())
        if len(clean_name)==0:
            return None
        elif clean_name[1]=="." and clean_name[3]==".":
            return clean_aname.replace(" ","_").title()
        elif clean_name.split(".",1)[0] in salutations:
            only_name = clean_name.split(".",1)[1]
            return only_name.replace(" ","_").title()
        else:
            return clean_name.replace(" ","_").title()

In [16]:
from pyspark.sql.functions import udf
name_udf  = udf(clean_name)

In [17]:
test_df.select('name',
 'father_name',
 'mother_name').show(10)

+--------------------+--------------------+-------------------+
|                name|         father_name|        mother_name|
+--------------------+--------------------+-------------------+
| Sabah Mati Alcasoas|             Prittam|      Kaavya Kanvar|
|  Rebia Raj Alcasoas|Jagdeep Raj Alcasoas|Araati Lata Bhomkar|
| Debiprasad Alcasoas|  Rebia Raj Alcasoas|Sabah Mati Alcasoas|
|      Payal Alcasoas|   Rebia Rajalcasoas|Sabah Mati Alcasoas|
|  Sarmistha Maisuria|   Rookmani Maisuria|    Kirthana Prabha|
|Sukhdev Thottuchalil|            Jism Sen|       Peyari Jeesm|
|         Rohana Doke|     Uma Prasad Doke|     Kanvar Supriti|
|       Katwa Parihar|        Baru Parihar|       Eeshani Vati|
|       Amari Parihar|      Kaatwa Parihar|        Rohana Doke|
|  Ramjee Dev Parihar|       Katwa Parihar|        Rohana Doke|
+--------------------+--------------------+-------------------+
only showing top 10 rows



In [18]:
test_df = test_df.withColumn("udf_name",name_udf(test_df["name"]))

In [19]:
test_df = test_df.withColumn("udf_f_name",name_udf(test_df["father_name"]))

In [20]:
test_df = test_df.withColumn("udf_m_name",name_udf(test_df["mother_name"]))

In [21]:
%time test_df.select("udf_name","udf_f_name","udf_m_name","name","father_name","mother_name").show(10)

+--------------------+--------------------+-------------------+--------------------+--------------------+-------------------+
|            udf_name|          udf_f_name|         udf_m_name|                name|         father_name|        mother_name|
+--------------------+--------------------+-------------------+--------------------+--------------------+-------------------+
| Sabah_Mati_Alcasoas|             Prittam|      Kaavya_Kanvar| Sabah Mati Alcasoas|             Prittam|      Kaavya Kanvar|
|  Rebia_Raj_Alcasoas|Jagdeep_Raj_Alcasoas|Araati_Lata_Bhomkar|  Rebia Raj Alcasoas|Jagdeep Raj Alcasoas|Araati Lata Bhomkar|
| Debiprasad_Alcasoas|  Rebia_Raj_Alcasoas|Sabah_Mati_Alcasoas| Debiprasad Alcasoas|  Rebia Raj Alcasoas|Sabah Mati Alcasoas|
|      Payal_Alcasoas|   Rebia_Rajalcasoas|Sabah_Mati_Alcasoas|      Payal Alcasoas|   Rebia Rajalcasoas|Sabah Mati Alcasoas|
|  Sarmistha_Maisuria|   Rookmani_Maisuria|    Kirthana_Prabha|  Sarmistha Maisuria|   Rookmani Maisuria|    Kirthana 

In [10]:
#df1.write.format("avro").save("file:///home/ns22/Desktop/test_ration_avro")

In [29]:
# option 1
from pyspark.sql import functions as F
renamed_df = test_df.select([F.col(col).alias(col.replace('_', ' ')) for col in test_df.columns])



In [32]:
# option 2
NewColumns=(column.replace(' ', '__') for column in renamed_df.columns)
new_df = renamed_df.toDF(*NewColumns)

In [39]:
# new_df = new_df.toDF(*(column.replace('__', '_') for column in new_df.columns))
# new_df.columns

In [26]:
import pandas as pd
df = pd.read_csv("file:///home/ns22/Desktop/rationcard/batch_1_500000_entities_23_dec_2021_rationcard.csv", sep=",")
df.head(5)

# in pandas 
# df.columns = [x.lower().replace("_"," ") for x in df.columns ] 


# in pyspark dataframe
#new_df = df.toDF(*(x.lower().replace("_"," ") for x in df.columns))
#new_df.columns

,updated_er_id,name,relationship_with_crd_owner,father_name,mother_name,gender,dob,age,member_id,address,state_name,district_name,rationcard_id
0,122311,Aeeni Afridi,self,Saadiq Bidhar,Mausami De Bidhar,F,12/01/1960,60,54910117654401,"Ekaparnika, Akshay Street, Sector 13, Nr. Rabp...",Kerala,Thrissur,549101176544
1,112311,Aphridi Bhai,husband,Kanive Sangdo,Aadab Prabha Ganti,M,07/07/1963,57,54910117654402,"Ekaparnika, Akshay Street, Sector 13, Nr. Rabp...",Kerala,Thrissur,549101176544
2,2311,Faria Kanive,daughter,Afridi Bhai,Aeeni Afridi,F,19/10/1989,31,54910117654403,"Ekaparnika, Akshay Street, Sector 13, Nr. Rabp...",Kerala,Thrissur,549101176544
3,1502311,Bhupinder Lata Mattathil,daughter,Aphridi Bhai,Aini Afridi,F,07/12/1994,26,54910117654404,"Ekaparnika, Akshay Street, Sector 13, Nr. Rabp...",Kerala,Thrissur,549101176544
4,12112312,Suprabha Shri Mankad,self,Maanik Gupta Mankad,Ekta,F,10/02/1962,58,25516389278801,"Kanchan, Bipasha Lane, Sector Number- 50, Opp....",Andhra Pradesh,East Godavari,255163892788


# Parquet files 

## Loading Data Programmatically

In [42]:
df= spark.read.csv("file:///home/ns22/Desktop/rationcard/batch_1_500000_entities_23_dec_2021_rationcard.csv",header=True, sep=",")
df = df.coalesce(1)
df.write.parquet("file:///home/ns22/Desktop/temp/test_parquet_ration.parquet")

In [17]:
parquet_file = spark.read.parquet("file:///home/ns22/Desktop/temp/test_parquet_ration.parquet")
parquet_file = parquet_file.coalesce(1)
parquet_file.createOrReplaceGlobalTempView("parquetFile")

In [18]:
parquet_file.show(5)

+-------------+--------------------+---------------------------+-------------------+------------------+------+----------+---+--------------+--------------------+--------------+-------------+-------------+
|updated_er_id|                name|relationship_with_crd_owner|        father_name|       mother_name|gender|       dob|age|     member_id|             address|    state_name|district_name|rationcard_id|
+-------------+--------------------+---------------------------+-------------------+------------------+------+----------+---+--------------+--------------------+--------------+-------------+-------------+
|       122311|        Aeeni Afridi|                       self|      Saadiq Bidhar| Mausami De Bidhar|     F|12/01/1960| 60|54910117654401|Ekaparnika, Aksha...|        Kerala|     Thrissur| 549101176544|
|       112311|        Aphridi Bhai|                    husband|      Kanive Sangdo|Aadab Prabha Ganti|     M|07/07/1963| 57|54910117654402|Ekaparnika, Aksha...|        Kerala|    

In [24]:
parquet_file.createOrReplaceTempView("parquetFile")

In [28]:
#spark.sql("SELECT /*+ COALESCE(3) */ * from parquetFile").show()

In [34]:
#spark.sql("SELECT /*+ REPARTITION(2, gender) */ * from parquetFile").show()

In [ ]:
all_states = spark.sql("select name, gender, age, state_name from parquetFile")
all_states.show(5)

+--------------------+------+---+--------------+
|                name|gender|age|    state_name|
+--------------------+------+---+--------------+
|        Aeeni Afridi|     F| 60|        Kerala|
|        Aphridi Bhai|     M| 57|        Kerala|
|        Faria Kanive|     F| 31|        Kerala|
|Bhupinder Lata Ma...|     F| 26|        Kerala|
|Suprabha Shri Mankad|     F| 58|Andhra Pradesh|
+--------------------+------+---+--------------+
only showing top 5 rows



In [ ]:
type(all_states)

pyspark.sql.dataframe.DataFrame

In [40]:
from pyspark.sql.functions import col

In [48]:
#parquet_file.repartition(4,col("state_name")).write.partitionBy(col("state_name")).mode(SaveMode.Append).saveAsTable("all_states_repartitioned_table")

#TypeError: Column is not iterable

In [31]:
spark.sql("""
    SELECT 
        state_name, 
        count(*) AS Number_of_person 
    FROM parquetFile 
    GROUP BY state_name 
    ORDER BY Number_of_person DESC""").show()

+----------------+----------------+
|      state_name|Number_of_person|
+----------------+----------------+
|      Tamil Nadu|           90442|
|   Uttar Pradesh|           72010|
|     Maharashtra|           69190|
|          Kerala|           62683|
|       Karnataka|           59416|
|  Andhra Pradesh|           54063|
|     West Bengal|           52792|
|         Gujarat|           45334|
|       Rajasthan|           43270|
|          Odisha|           40026|
|           Bihar|           37998|
|  Madhya Pradesh|           33918|
|       Telangana|           29198|
|           Assam|           24973|
|          Punjab|           23385|
|Himachal Pradesh|           18863|
|       Jharkhand|           16937|
|         Haryana|           13254|
|     Uttarakhand|           12591|
|     Chattisgarh|           11481|
+----------------+----------------+
only showing top 20 rows



## Compress further from parquet to snappy

In [59]:
parquet_file.write.option("compression","snappy").parquet("file:///home/ns22/Desktop/temp/snappy")

In [67]:
test_df.count()

15219768

In [68]:
test_df = test_df.coalesce(1)
test_df.write.option("compression","snappy").parquet("file:///home/ns22/Desktop/test_ration_parquet_new/snappy")
# More details about snappy compression
# http://boristyukin.com/is-snappy-compressed-parquet-file-splittable/

In [69]:
test_snappy_df = spark.read.parquet("file:///home/ns22/Desktop/test_ration_parquet_new/snappy")

In [73]:
test_snappy_df.filter(test_snappy_df.state_name == "Jharkhand").show()

+-------------+--------------------+---------------------------+--------------------+--------------------+------+----------+---+--------------+--------------------+----------+--------------+-------------+
|updated_er_id|                name|relationship_with_crd_owner|         father_name|         mother_name|gender|       dob|age|     member_id|             address|state_name| district_name|rationcard_id|
+-------------+--------------------+---------------------------+--------------------+--------------------+------+----------+---+--------------+--------------------+----------+--------------+-------------+
|     12233168|              Iditri|                       self|         Yudhbir Ram|     Yudhbir Bibhuti|     F|07/04/1972| 48|39590910826901|X/61, 7 Floor, Va...| Jharkhand|West Singhbhum| 395909108269|
|     11233168|    Yusuf Dasa Palit|                    husband|          Aamir Dasa|     Arwa Shri Aamir|     M|14/02/1981| 39|39590910826902|X/61, 7 Floor, Va...| Jharkhand|West 

# Read ElasticSearch index into pyspark dataframe 

In [2]:
from itertools import count
import elasticsearch 
from elasticsearch.helpers import scan
import pandas as pd
# import elastic_mapping
import json
from pyspark.sql import SparkSession


spark = SparkSession.builder.appName("elastc_test").getOrCreate() 

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/11 11:26:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [81]:
#pip install elasticsearch

     |████████████████████████████████| 382 kB 214 kB/s eta 0:00:01
  Using cached elastic_transport-8.1.2-py3-none-any.whl (59 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
import elasticsearch


es = elasticsearch.Elasticsearch([{'host': '192.168.181.124', 'port': 9200, 'scheme': 'http'}], max_retries=10,
                                 retry_on_timeout=True)
elastic_index_name = "eva_trends_trial"

# Elastic Connection
def elastic_connection():
    if es.ping():
        print("ElasticSearch Connect Successfully")
    else:
        print('Awww it could not connect!')

In [4]:
elastic_connection()

ElasticSearch Connect Successfully


/tmp/ipykernel_305380/65089427.py:10: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  if es.ping():


In [15]:
def get_query(size):
    
    query = {
            "query": {
                "match_all": {}
                },
		    "sort": [
		    {
		      "ingestedtime": {
			        "order": "desc"
		      		}
    			}
  		    ]
        }


    rel = scan(client=es,             
                query=query,
               size=100,
                scroll='1m',
                index=elastic_index_name,
                raise_on_error=True,
                preserve_order=False,
                clear_scroll=True)
    

    
    # Keep response in a list.
    result = list(rel)
    length = len(result)
    doc_count = {}
    # doc_count['total_count'] = length
    print("count of documents: ", length)
    temp = []
    # temp.append(doc_count)
    # We need only '_source', which has all the fields required.
    # This elimantes the elasticsearch metdata like _id, _type, _index.
    for hit in result:
        temp.append(hit['_source'])
        if len(temp) > size:
            # print("temp : " ,temp)
            return temp   
    return temp



data = get_query(100)
print("Data Fetched")

df = spark.createDataFrame(data)
print("Completed")
print(df.columns)
print(df.select('articletime', 'baseurl', 'count_int', 'count_str', 'country', 'hostname').show(10))

/tmp/ipykernel_305380/535923404.py:29: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  result = list(rel)


count of documents:  184957
Data Fetched
Completed
['articletime', 'baseurl', 'count_int', 'count_str', 'country', 'hostname', 'ingestedtime', 'language', 'rank', 'resource_type', 'title', 'category', 'image_link', 'news_source', 'related_links']
+--------------------+--------------------+---------+---------+--------+--------------------+
|         articletime|             baseurl|count_int|count_str| country|            hostname|
+--------------------+--------------------+---------+---------+--------+--------------------+
|2022-07-05T15:04:25Z|https://trends24.in/|        0|       NA|Pakistan|            Trends24|
|2022-07-05T17:56:04Z|https://trends.go...|     null|     null|   India|Google Realtime T...|
|2022-07-05T15:04:20Z|https://trends24.in/|        0|       NA|Pakistan|            Trends24|
|2022-07-05T15:03:56Z|https://trends24.in/|    43000|      43K|Pakistan|            Trends24|
|2022-07-05T19:56:04Z|https://trends.go...|     null|     null|   India|Google Realtime T...|
|

## Modified..Read Elastic search index data in pyspark dataframe

In [63]:
def get_query():
    
    query = {
            "query": {
                "match_all": {}
                },
		    "sort": [
		    {
		      "ingestedtime": {
			        "order": "desc"
		      		}
    			}
  		    ]
        }


    rel = es.search(index=elastic_index_name,body=query,size=11)
    

    length = rel['hits']['total']['value']
    # Keep response in a list.
    # result = list(rel)
    # length = len(result)
    doc_count = {}
    # doc_count['total_count'] = length
    print("count of documents: ", length)
    temp = []
    temp.append(doc_count)
    # We need only '_source', which has all the fields required.
    # This elimantes the elasticsearch metdata like _id, _type, _index.
    for hit in rel['hits']['hits']:
        temp.append(hit['_source'])   
    return temp



data = get_query()
print("Data Fetched")

df = spark.createDataFrame(data)
print("Completed")
print(df.columns)
print(df.select('articletime', 'baseurl', 'count_int', 'count_str', 'country', 'hostname').show(10))
# columns = df.columns



count of documents:  10000
Data Fetched
Completed
['articletime', 'baseurl', 'count_int', 'count_str', 'country', 'hostname', 'ingestedtime', 'language', 'rank', 'resource_type', 'title']
+--------------------+--------------------+---------+---------+--------------+--------+
|         articletime|             baseurl|count_int|count_str|       country|hostname|
+--------------------+--------------------+---------+---------+--------------+--------+
|                null|                null|     null|     null|          null|    null|
|2022-07-07T12:38:31Z|https://trends24.in/|   789000|     789K|United Kingdom|Trends24|
|2022-07-07T12:38:31Z|https://trends24.in/|   110000|     110K|United Kingdom|Trends24|
|2022-07-07T12:38:31Z|https://trends24.in/|    88000|      88K|United Kingdom|Trends24|
|2022-07-07T12:38:31Z|https://trends24.in/|    77000|      77K|United Kingdom|Trends24|
|2022-07-07T12:38:31Z|https://trends24.in/|    51000|      51K|United Kingdom|Trends24|
|2022-07-07T12:38:31

/tmp/ipykernel_305380/4220753924.py:17: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  rel = es.search(index=elastic_index_name,body=query,size=11)
/tmp/ipykernel_305380/4220753924.py:17: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  rel = es.search(index=elastic_index_name,body=query,size=11)


In [67]:
# Deletes the record having all null values
#df.na.drop("all").show(10,truncate=False)

In [84]:
# convert pyspark df to pandas df , use iloc and return to pyspark df
ps_df = df.toPandas()
new_df = ps_df.iloc[1:]
sparkDF=spark.createDataFrame(new_df)
sparkDF.show()

+--------------------+--------------------+---------+---------+--------------+--------+--------------------+--------+----+-------------+--------------+
|         articletime|             baseurl|count_int|count_str|       country|hostname|        ingestedtime|language|rank|resource_type|         title|
+--------------------+--------------------+---------+---------+--------------+--------+--------------------+--------+----+-------------+--------------+
|2022-07-07T12:38:31Z|https://trends24.in/| 789000.0|     789K|United Kingdom|Trends24|2022-07-07T17:38:31Z|      en| 1.0|       Trends|         Boris|
|2022-07-07T12:38:31Z|https://trends24.in/| 110000.0|     110K|United Kingdom|Trends24|2022-07-07T17:38:31Z|      sl| 2.0|       Trends|          Gove|
|2022-07-07T12:38:31Z|https://trends24.in/|  88000.0|      88K|United Kingdom|Trends24|2022-07-07T17:38:31Z|      tl| 3.0|       Trends|         #PMQs|
|2022-07-07T12:38:31Z|https://trends24.in/|  77000.0|      77K|United Kingdom|Trends24|2

In [54]:
# df.createOrReplaceTempView("my_table")
# spark.sql("select * from my_table limit 5").show()
# spark.sql("DELETE TOP(1) FROM my_table").show()

ParseException: 
Syntax error at or near 'TOP': missing 'FROM'(line 1, pos 7)

== SQL ==
DELETE TOP(1) FROM my_table
-------^^^


# Mapping in Spark using broadcast variables

## PySpark RDD Broadcast variable example

In [74]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

states = {"NY":"New York", "CA":"California", "FL":"Florida"}
broadcastStates = spark.sparkContext.broadcast(states)

data = [("James","Smith","USA","CA"),
    ("Michael","Rose","USA","NY"),
    ("Robert","Williams","USA","CA"),
    ("Maria","Jones","USA","FL")
  ]

rdd = spark.sparkContext.parallelize(data)

def state_convert(code):
    return broadcastStates.value[code]

result = rdd.map(lambda x: (x[0],x[1],x[2],state_convert(x[3]))).collect()
print(result)

22/07/08 14:30:21 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


[('James', 'Smith', 'USA', 'California'), ('Michael', 'Rose', 'USA', 'New York'), ('Robert', 'Williams', 'USA', 'California'), ('Maria', 'Jones', 'USA', 'Florida')]


## PySpark DataFrame Broadcast variable example


In [75]:

import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

states = {"NY":"New York", "CA":"California", "FL":"Florida"}
broadcastStates = spark.sparkContext.broadcast(states)

data = [("James","Smith","USA","CA"),
    ("Michael","Rose","USA","NY"),
    ("Robert","Williams","USA","CA"),
    ("Maria","Jones","USA","FL")
  ]

columns = ["firstname","lastname","country","state"]
df = spark.createDataFrame(data = data, schema = columns)
df.printSchema()
df.show(truncate=False)

def state_convert(code):
    return broadcastStates.value[code]

result = df.rdd.map(lambda x: (x[0],x[1],x[2],state_convert(x[3]))).toDF(columns)
result.show(truncate=False)


root
 |-- firstname: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- country: string (nullable = true)
 |-- state: string (nullable = true)

+---------+--------+-------+-----+
|firstname|lastname|country|state|
+---------+--------+-------+-----+
|James    |Smith   |USA    |CA   |
|Michael  |Rose    |USA    |NY   |
|Robert   |Williams|USA    |CA   |
|Maria    |Jones   |USA    |FL   |
+---------+--------+-------+-----+

+---------+--------+-------+----------+
|firstname|lastname|country|state     |
+---------+--------+-------+----------+
|James    |Smith   |USA    |California|
|Michael  |Rose    |USA    |New York  |
|Robert   |Williams|USA    |California|
|Maria    |Jones   |USA    |Florida   |
+---------+--------+-------+----------+



# SQL in Spark

In [89]:
spark.sql("create database new_db;")

DataFrame[]

In [97]:
# spark.sql("""
# CREATE TABLE CUSTOMERS(
#    ID   INT              NOT NULL,
#    NAME VARCHAR (20)     NOT NULL,
#    AGE  INT              NOT NULL,
#    ADDRESS  CHAR (25) ,
#    SALARY   DECIMAL (18, 2)
# );""")

In [92]:
spark.sql("show databases;").show()

+---------+
|namespace|
+---------+
|  default|
|   new_db|
+---------+



In [100]:
test_file = spark.read.parquet("file:///home/ns22/Desktop/temp/test_parquet_ration.parquet")
test_file = test_file.coalesce(1)
test_file.createOrReplaceTempView("testTable")

In [102]:
spark.sql("""
SELECT * FROM testTable
""").show(1)

+-------------+------------+---------------------------+-------------+-----------------+------+----------+---+--------------+--------------------+----------+-------------+-------------+
|updated_er_id|        name|relationship_with_crd_owner|  father_name|      mother_name|gender|       dob|age|     member_id|             address|state_name|district_name|rationcard_id|
+-------------+------------+---------------------------+-------------+-----------------+------+----------+---+--------------+--------------------+----------+-------------+-------------+
|       122311|Aeeni Afridi|                       self|Saadiq Bidhar|Mausami De Bidhar|     F|12/01/1960| 60|54910117654401|Ekaparnika, Aksha...|    Kerala|     Thrissur| 549101176544|
+-------------+------------+---------------------------+-------------+-----------------+------+----------+---+--------------+--------------------+----------+-------------+-------------+
only showing top 1 row



In [114]:
spark.sql("""
SELECT updated_er_id FROM testTable limit 3 
""").show()

+-------------+
|updated_er_id|
+-------------+
|       122311|
|       112311|
|         2311|
+-------------+



In [112]:
spark.sql("""
SELECT gender, count(*) FROM testTable GROUP BY gender
""").show()

+------+--------+
|gender|count(1)|
+------+--------+
|     F|  464297|
|     M|  384088|
|     T|     438|
+------+--------+



# Print All available dataframes in current spark session

In [119]:
from pyspark.sql import DataFrame
print([k for (k,v) in globals().items() if isinstance(v, DataFrame)])

['_', '__', '___', 'df', 'parquet_file', 'all_states', 'sparkDF', '_87', '_89', '_91', 'test_file']


In [122]:
spark.catalog.listDatabases()

[Database(name='default', description='default database', locationUri='file:/home/ns22/Downloads/new_validation/spark-warehouse'),
 Database(name='new_db', description='', locationUri='file:/home/ns22/Downloads/new_validation/spark-warehouse/new_db.db')]